In [1]:
%reload_ext autotime
import pandas as pd
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
from pprint import pprint
import json5 as json # This is a more forgiving JSON parser that can handle comments, single quotes, and trailing commas
from glob import glob
from tqdm import tqdm
files = glob("videos/*.json")
print(len(files))

402
time: 513 ms (started: 2025-10-20 10:34:28 +13:00)


In [11]:
pd.to_timedelta("00:" + pd.read_csv("googlesearch.csv").duration).describe()

count                          470
mean     0 days 00:00:53.255319148
std      0 days 00:00:38.388579994
min                0 days 00:00:01
25%                0 days 00:00:20
50%                0 days 00:00:52
75%                0 days 00:01:18
max                0 days 00:03:00
Name: duration, dtype: object

time: 31.3 ms (started: 2025-10-20 10:40:18 +13:00)


In [2]:
from transformers import Qwen3OmniMoeForConditionalGeneration, Qwen3OmniMoeProcessor
from qwen_omni_utils import process_mm_info
import torch
MODEL_PATH = "Qwen/Qwen3-Omni-30B-A3B-Instruct"
# Uses about 78GB VRAM
model = Qwen3OmniMoeForConditionalGeneration.from_pretrained(
    MODEL_PATH,
    dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2",
)
model.disable_talker()
processor = Qwen3OmniMoeProcessor.from_pretrained(MODEL_PATH)

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_interleaved', 'mrope_section', 'interleaved'}
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section', 'interleaved'}
You are attempting to use Flash Attention 2 without specifying a torch dtype. This might lead to unexpected behaviour


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


time: 2min 10s (started: 2025-10-20 10:34:28 +13:00)


In [3]:
with open("videos/Mendisiplinkan anak tanpa perlu marah-marah #austinkoh #timeout #pare... [7098274440739360026].info.json") as f:
    data = json.load(f)
display(pd.json_normalize(data))
print(data["description"])

,id,formats,channel,channel_id,uploader,uploader_id,channel_url,uploader_url,track,artists,duration,title,description,timestamp,view_count,like_count,repost_count,comment_count,thumbnails,webpage_url,webpage_url_basename,webpage_url_domain,extractor,extractor_key,thumbnail,display_id,fulltitle,duration_string,upload_date,artist,epoch,ext,vcodec,acodec,format_id,tbr,quality,preference,filesize,width,height,url,protocol,video_ext,audio_ext,resolution,dynamic_range,aspect_ratio,cookies,format,_type,http_headers.User-Agent,http_headers.Accept,http_headers.Accept-Language,http_headers.Sec-Fetch-Mode,http_headers.Referer,_version.version,_version.release_git_head,_version.repository
0,7098274440739360026,"[{'ext': 'mp4', 'vcodec': 'h264', 'acodec': 'aac', 'format_id': 'download', 'url': 'https://v16-...",Austin Koh & Family,MS4wLjABAAAAe6pWn55onzo6jSYIjHAjHNLtHgckCmmRfxELU9R5ZGrXMZHo0FJgSdjr4BEEJlgy,austinkohandfamily,6766875852917130241,https://www.tiktok.com/@MS4wLjABAAAAe6pWn55onzo6jSYIjHAjHNLtHgckCmmRfxELU9R5ZGrXMZHo0FJgSdjr4BEE...,https://www.tiktok.com/@austinkohandfamily,《钢琴与弦乐》雨后香气,[没有发光的金],163,Mendisiplinkan anak tanpa perlu marah-marah #austinkoh #timeout #pare...,Mendisiplinkan anak tanpa perlu marah-marah #austinkoh #timeout #parenting #tipsparenting #fyp #...,1652695807,6400000,605700,8863,4243,"[{'id': 'dynamicCover', 'url': 'https://p19-common-sign-useastred.tiktokcdn-eu.com/tos-useast2a-...",https://www.tiktok.com/@austinkohandfamily/video/7098274440739360026,7098274440739360026,tiktok.com,TikTok,TikTok,https://p19-common-sign-useastred.tiktokcdn-eu.com/tos-useast2a-p-0037-aiso/422adc82c79a4716804f...,7098274440739360026,Mendisiplinkan anak tanpa perlu marah-marah #austinkoh #timeout #pare...,2:43,20220516,没有发光的金,1760751674,mp4,h265,aac,bytevc1_720p_476746-1,476,2,-1,9755307,720,1280,https://v19-webapp-prime.tiktok.com/video/tos/useast2a/tos-useast2a-pve-0037-aiso/b8d5c87f8be748...,https,mp4,none,720x1280,SDR,0.56,ttwid=1%7CyrOk4UQGVTrtnRSf0ZZLXrEIrt1DGfsiF8a8lvfQReQ%7C1760751016%7Cc96afef07b445b56f0f03c975fc...,bytevc1_720p_476746-1 - 720x1280,video,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0....","text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","en-us,en;q=0.5",navigate,https://www.tiktok.com/@austinkohandfamily/video/7098274440739360026,2025.10.14,a98e7f9f58a9492d2cb216baa59c890ed8ce02f3,yt-dlp/yt-dlp


Mendisiplinkan anak tanpa perlu marah-marah #austinkoh #timeout #parenting #tipsparenting #fyp #viral2022 #viral #fypシ #bayilucu #serunyaditiktok #parentingtips #parentingindonesia #bayigemes #bayitiktok #GoyangGemasMochiBaby
time: 323 ms (started: 2025-10-20 10:36:39 +13:00)


In [4]:
def get_prompt(data):
    return f"""This is a video downloaded from {data['extractor']}. Here's the description of the video: {data['description']}.

        The creator of the video is {data.get('channel', 'an unknown channel')} ({data.get('uploader', 'an unknown uploader')})
        It has {data.get('like_count', 'an unknown number of')} likes, {data.get('view_count', 'an unknown number of')} views, and {data.get('comment_count', 'an unknown number of')} comments.
        Taking into account this description, and the video, extract the following information, in JSON format:
        description: What is happening in the video? Provide a detailed description of the actions, context, and any notable elements present in the video.
        transcript: If there is any spoken content in the video, transcribe it into English. If there is no spoken content, indicate "No spoken content".
        tone: What is the overall tone or mood of the video? Is it humorous, serious, educational, emotional, etc.?
        timeout: Is this video talking about timeout for children (as a parenting punishement strategy)? a boolean true or false
        language: What language is this video in?
        marketing: Is this video promoting or advertising any product, service, brand, or organization? If so, what is it? Otherwise, indicate "No marketing content".
        ASD: Is this video specifically targeted towards individuals with Autism Spectrum Disorder (ASD) or does it contain content that is particularly relevant or beneficial for this audience? a boolean true or false
        ADHD: Is this video specifically targeted towards individuals with Attention Deficit Hyperactivity Disorder (ADHD) or does it contain content that is particularly relevant or beneficial for this audience? a boolean true or false
        anxiety: Is this video specifically targeted towards individuals dealing with anxiety or does it contain content that is particularly relevant or beneficial for this audience? a boolean true or false
        job: For the main speaker, what is their job or profession? If it is not mentioned in the video, indicate "No job information". A comma separated string, one or more of the following: therapist, psychologist, pediatrician, doctor, nurse, teacher, professor, social worker, counselor, coach, influencer, content creator?
        sentiment: Does this video recommend timeout, discourage it, or is it neutral? One of negative, neutral or positive
        criticism: If the video is critical of timeout, what are the main criticisms mentioned? A comma separated string, one or more of the following:
            Concerns about physical safety – child
            Concerns about emotional or social development or wellbeing - child
            Concerns about relational quality – parent-child
            Concerns about practicality or usability
            Concerns about litigation
        alternative_strategies: Does the video mention any alternative strategies to timeout? If so, what are they? A comma separated string. If no alternatives are mentioned, indicate "No alternative strategies mentioned".
        usefulness: Rate the overall usefulness of the video on a scale from 1 to 10, where 1 is not useful at all and 10 is extremely useful.
        misleading: Rate the extent to which the video contains misleading or inaccurate information on a scale from 1 to 10, where 1 is not misleading at all and 10 is extremely misleading.
        quality: Rate the overall quality of the video on a scale from 1 to 10, where 1 is very poor quality and 10 is excellent quality.
        personal_experience: Does the speaker mention any personal experience with timeout? If so, briefly summarize it.

        Do not include comments in your JSON response. Only respond with the JSON object. Make sure the JSON is valid
    """
print(get_prompt(data))

This is a video downloaded from TikTok. Here's the description of the video: Mendisiplinkan anak tanpa perlu marah-marah #austinkoh #timeout #parenting #tipsparenting #fyp #viral2022 #viral #fypシ #bayilucu #serunyaditiktok #parentingtips #parentingindonesia #bayigemes #bayitiktok #GoyangGemasMochiBaby.

        The creator of the video is Austin Koh & Family (austinkohandfamily)
        It has 605700 likes, 6400000 views, and 4243 comments.
        Taking into account this description, and the video, extract the following information, in JSON format:
        description: What is happening in the video? Provide a detailed description of the actions, context, and any notable elements present in the video.
        transcript: If there is any spoken content in the video, transcribe it into English. If there is no spoken content, indicate "No spoken content".
        tone: What is the overall tone or mood of the video? Is it humorous, serious, educational, emotional, etc.?
        timeout: 

In [5]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "videos/Mendisiplinkan anak tanpa perlu marah-marah #austinkoh #timeout #pare... [7098274440739360026].mp4",
                "max_pixels": 360 * 420,
            },
            {
                "type": "text",
                "text": get_prompt(data)
            },
        ],
    }
]
# Set whether to use audio in video
USE_AUDIO_IN_VIDEO = True

# Preparation for inference
text = processor.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
audios, images, videos = process_mm_info(messages, use_audio_in_video=USE_AUDIO_IN_VIDEO)
inputs = processor(text=text, 
                   audio=audios, 
                   images=images, 
                   videos=videos, 
                   return_tensors="pt", 
                   padding=True, 
                   use_audio_in_video=USE_AUDIO_IN_VIDEO)
inputs = inputs.to(model.device).to(model.dtype)

# Inference: Generation of the output text and audio
text_ids, audio = model.generate(**inputs, 
                                 thinker_return_dict_in_generate=True,
                                 use_audio_in_video=USE_AUDIO_IN_VIDEO)

text = processor.batch_decode(text_ids.sequences[:, inputs["input_ids"].shape[1] :],
                              skip_special_tokens=True,
                              clean_up_tokenization_spaces=False)[0]
print(text)

/mnt/timeout/.venv/lib/python3.12/site-packages/qwen_omni_utils/v2_5/audio_process.py:85: UserWarning: PySoundFile failed. Trying audioread instead.
  librosa.load(
/mnt/timeout/.venv/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
qwen-vl-utils using torchcodec to read video.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


```json
{
  "description": "A young child is sitting on a bed, covered with a grey comforter. An adult's hand holds a blue and light-blue bento-style lunchbox in front of the child. The child, wearing a patterned short-sleeve shirt, looks at the lunchbox and then at the adult, speaking and gesturing. The adult places the lunchbox on the bed in front of the child. The child then begins to talk animatedly, using various hand gestures, including clapping, waving, and making a 'shushing' motion. The child continues to talk while holding the lunchbox, looking around the room and at the adult.",
  "transcript": "Mama: Kenapa bekalnya nggak dimakan? Jadi makan apa di sekolah?\nAnak: Makan lollipop.\nMama: Makan lollipop? Siapa yang kasih?\nAnak: Maxwell.\nMama: Maxwell itu siapa? Yang ngasih?\nAnak: Iya, kawannya Austin.\nMama: Kawan Austin? Dia kasih Austin lollipop?\nAnak: Iya.\nMama: Kok makan lollipop? Karena Austin good?\nAnak: Iya.\nMama: Banyak lollipop yang dimakan?\nAnak: Banyak.\nMa